# Mini-project n° 2 – Conditional VAEs

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


## Listes des hyperparamètres

Les définir + justifier les valeurs

Les hyperparamètres de notre modèle sont : 

- batch size = Le nombre d'échantillons sur lesquels le modèle apprend avant de mettre à jour ses paramètres internes (poids du réseau...)
- epochs = Le nombre de fois où l'entièreté du jeu de données est utilisée pour l'apprentissage. Une valeur trop faible peut conduire à du sous-apprentissage et au contraire une valeur trop élevée à du surapprentissage
- latent_dim = La taille de l'espace latent
- learning_rate = Le taux d'apprentissage. Il contrôle la vitesse à laquelle le modèle apprend.
- kl_weights = La liste des poids pour la divergence KL (Kullback-Leibler)
- beta = Ajuste l'importance de la divergence dans la fonction de perte

La fonction de perte utilisée ci-après est définie de la manière suivante :

Loss = BCE + beta * KLD

Le premier terme est un terme de reconstruction : il mesure à quel point le réseau reconstruit correctement les données d’origine à partir de l’espace latent.
Le second terme est un terme de régularisation : il mesure à quel point la distribution de l'espace latent se rapproche d'une loi normale centrée réduite.

Ainsi, 3 cas sont à distinguer concernant le paramètre beta :

- beta = 1 : On accorde autant d'importance à la reconstruction qu'à la régularisation.
- beta > 1 : On donne davantage de poids à la régularisation. On fait en sorte que la distribution de l'espace latent se rapproche au plus d'une gaussienne centrée réduite. Cependant, cela peut diminuer la capacité de reconstruction du modèle.
- beta < 1 : On donne davantage de poids à la reconstruction. Le modèle apprend ainsi à capturer plus de détails dans l'image. Cependant, cela peut augmenter le risque de surapprentissage.

In [2]:
batch_size = 128
epochs = 30
latent_dim = 2
learning_rate = 1e-3
kl_weights = [1, 10, 100]
beta = 1

## Chargement des données

In [3]:
# Data loading
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='../../data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='../../data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1129)>



100%|██████████| 9912422/9912422 [00:18<00:00, 540367.98it/s] 


Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1129)>



100%|██████████| 28881/28881 [00:00<00:00, 323325.98it/s]


Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1129)>



100%|██████████| 1648877/1648877 [00:05<00:00, 310657.61it/s]


Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1129)>



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw



## Création d'un modèle d'AutoEndodeur Variationnel Conditionnel (Conditionnal VAE)

Le modèle de VAE conditionnel utilisé par la suite se construit de la manière suivante :

- **Un Encodeur** : 
    - Une couche de convolution composée de 32 filtres, un noyau de taille 4, un stride de 2 and un padding de 1.
    - Une couche BatchNorm qui conserve le même nombre de caractérstiques.
    - Une fonction d'activation ReLu.
    - Une couche de convolution composée de 64 filtres, un noyau de taille 4, un stride de 2 and un padding de 1.
    - Une couche BatchNorm qui conserve le même nombre de caractérstiques.
    - Une fonction d'activation ReLu.
    - Une couche de convolution composée de 128 filtres, un noyau de taille 3, un stride de 2 and un padding de 1..
    - Une couche BatchNorm qui conserve le même nombre de caractérstiques.
    - Une fonction d'activation ReLu.

- **Un Espace Latent** :

- **Un Décodeur.** 
    - Une couche de déconvolution composée de 64 filtres, un noyau de taille 3, un stride de 2 and un padding de 1.
    - Une couche BatchNorm qui conserve le même nombre de caractérstiques
    - Une fonction d'activation ReLu
    - Une couche de déconvolution composée de 32 filtres, un noyau de taille 4, un stride de 2 and un padding de 1.
    - Une couche BatchNorm qui conserve le même nombre de caractérstiques
    - Une fonction d'activation ReLu
    - Une couche de déconvolution composée de 1 filtre, un noyau de taille 4, un stride de 2 and un padding de 1.
    - Une fonction d'activation Sigmoïde

**Au niveau de l'encodeur :**

Pour chacune des couches de convolution :
- Un filtre (ou noyau) est une matrice de poids qui parcourt l’image d’entrée pour extraire des caractéristiques spécifiques (de manière générale comme des contours ou des textures). La sortie de la couche est obtenue à partir d'une convolution entre l'objet en entrée et le filtre. Le paramètre kernel_size désigne la taille de la matrice.
- Un filtre apprend UNE caractéristique de l’image donc, plus il y a de filtres, plus le modèle retient de caractéristiques.
- Le stride indique de combien de pixels le filtre se déplace à chaque étape lorsqu’il parcourt l’image d’entrée (d'abord horizontalement puis verticalement). 
- Le padding consiste à ajouter des pixels autour de l'image pour essayer de préserver l'information sur les bords de l'image. Un padding de 1 signfie que l'on ajoute une rangée de pixel tout autour de l'image

L’augmentation progressive du nombre de filtres dans les couches de convolution permet de capturer des caractéristiques de plus en plus complexes des données d’entrée : les couches proches de l'entrée capturent des motifs de base et au fur et à mesure que l’on avance dans le réseau, on veut que chaque couche représente des détails plus sophistiqués. Avoir plus de filtres permet à chaque couche de détecter des motifs plus variés et spécifiques.

La couche BatchNorm permet de normaliser les sorties d'une couche de réseau lors de l'apprentissage.

**Au niveau de l'espace latent :**

Il est composé de deux couches parallèles dites "entièrement connectées" (en d'autres termes tous les neurones sont connectés à ceux de la couche précédente, qui est ici la sortie de l'encodeur E(x)).

- La première couche estime un vecteur moyenne µ(x)
- La seconde couche estime un vecteur de log-variance $\log\sigma^2(x)$

Une dernière couche entièrement connectée utilise alors ces informations pour renvoyer le vecteur de l'espace latent z slon une distribution gaussienne : $z(x)\sim\mathcal{N}(\mu(x),Diag(\sigma(x)))$ 

**Au niveau du décodeur :**

La fonction d'activation utilisée en sortie est la fonction Sigmoïde. Elle renvoie des valeurs comprises entre 0 et 1 , ce qui est utile puisqu' avec ce réseau on cherche à générer des images en noir et blanc. Les résultats obtenus permettent donc directement d'obtenir des nuances de gris.




c sera à expliciter = condition

In [4]:
class ConvVAE(nn.Module):
    def __init__(self, latent_dim=10):
        super(ConvVAE, self).__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        # Fully connected layers for mean and log variance
        self.fc_mu = nn.Linear(128 * 4 * 4, latent_dim) # (128*4*4) est la taille de sortie de la couche précédente
        self.fc_logvar = nn.Linear(128 * 4 * 4, latent_dim) 
        self.fc_decode = nn.Linear(latent_dim, 128 * 4 * 4) 
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
    
    def encode(self, x, c):
        x = self.encoder(x)
        x = torch.cat((x,c),dim=1)
        x = x.view(-1, 128 * 4 * 4) # Flatten the output of the convolutional layers
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar
    

    def latent_sample(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z, c):
        x = torch.cat((x,c),dim=1)
        x = self.fc_decode(z)
        x = x.view(-1, 128, 4, 4)  # Reshape to (128, 4, 4) for the decoder
        x = self.decoder(x)
        return x

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.latent_sample(mu, logvar)
        return self.decode(z, c), mu, logvar

## Création de la fonction de perte (loss function)

In [5]:
def loss_function(recon_x, x, mu, logvar, beta=1):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum') # terme de reconstruction 
    # on utilise binary entropy pour forcer les contrastes (sinon on pourrait travailler avec MSE)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # terme de regularisation (on veut se rapprocher loi normale)
    return BCE + beta * KLD # l'objectif est d'avoir un bon compromis entre reconstruction et regularisation

## Apprentissage du modèle de VAE conditionnel

In [6]:
# Initialize the VAE model and the Adam optimizer
vae = ConvVAE(latent_dim=latent_dim)
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)

# Train the model for the given number of epochs
# At the end of each epoch, print the training loss
for epoch in range(1, epochs + 1):
    vae.train()
    running_loss = 0.0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch}, Training loss: {epoch_loss:.4f}')

TypeError: forward() missing 1 required positional argument: 'c'

## Génération des images

In [ ]:
def generate_sample(num_samples=10, digit):
# digit est la valeur conditionelle
    vae.eval()
    with torch.no_grad():

        # Échantillonage selon une loi normale
        z = torch.randn(num_samples, latent_dim).to(device)  # Sample random latent vectors

        # On ajoute la condition ie la valeur que l'on veut générer
        condition = torch.zeros(100,10, dtype=int)
        condition[:,digit] = 1 # indique que c'est la valeur digit que l'on cherche à générer

        # On génère l'image
        samples = vae.decode(z, condition) # Decode the latent vectors
        samples = samples.cpu().view(num_samples, 1, 28, 28) # Reshape the samples

        fig, ax = plt.subplots(1, num_samples, figsize=(15, 2))
        for i in range(num_samples):
            ax[i].imshow(samples[i].squeeze(0), cmap='gray')
            ax[i].axis('off')
        plt.show()

generate_sample()

## Visualisation de l'espace latent

def loss_function(recon_x, x, mu, logvar, beta=1):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + beta * KLD, BCE, KLD

In [ ]:
# Training and plotting function
def train_and_plot(kl_weight):
    model = ConvVAE(latent_dims).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model.train()
    train_losses = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        bce_loss = 0
        kld_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            x_recon, mu, logvar = model(data)
            loss, bce, kld = loss_function(x_recon, data, mu, logvar, kl_weight)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            bce_loss += bce.item()
            kld_loss += kld.item()
        
        average_loss = epoch_loss / len(train_loader.dataset)
        average_bce = bce_loss / len(train_loader.dataset)
        average_kld = kld_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}: Average Loss: {average_loss:.4f}, BCE: {average_bce:.4f}, KLD: {average_kld:.4f}')
    
    # Plot latent space
    plot_latent_space(model, kl_weight)

# Function to plot latent space
def plot_latent_space(model, kl_weight):
    model.eval()
    with torch.no_grad():
        test_loader = DataLoader(dataset=test_dataset, batch_size=10000, shuffle=False)
        data, labels = next(iter(test_loader))
        data = data.to(device)
        mu, logvar = model.encode(data)
        z = mu  # For visualization, we use the mean
        z = z.cpu().numpy()
        labels = labels.numpy()
        
        plt.figure(figsize=(8,6))
        scatter = plt.scatter(z[:, 0], z[:, 1], c=labels, cmap='tab10', alpha=0.7)
        plt.colorbar(scatter, ticks=range(10))
        plt.clim(-0.5, 9.5)
        plt.title(f'Latent Space with KL Weight = {kl_weight}')
        plt.xlabel('Z1')
        plt.ylabel('Z2')
        plt.show()

# Run training and plotting for different KL weights
for kl_weight in kl_weights:
    print(f'\nTraining VAE with KL Weight = {kl_weight}')
    train_and_plot(kl_weight)